# Building a Graph Neural Network from Scratch
> Author: Nathan Albin (albin@k-state.edu)\
> Updated: 2024-05-02

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, classification_report

import torch
import torch.nn as nn
torch.set_default_dtype(torch.float64)

import torch_geometric.nn as tgnn

from modified_cora import load_data

# Introduction

We are building this notebook in class as a way to see how graph neural networks are built step-by-step. Although we will eventually use the PyG library, my hope is that building a network by hand will help remove some of the mystery surrounding how these networks operate.

# The data

The next few cells load the (modified) Cora dataset and print a few rows of each dataframe.

In [4]:
content_df, cites_df = load_data()

citation data is in cora.tgz



In [5]:
content_df.head()

,paper_id,word_0000,word_0001,word_0002,word_0003,word_0004,word_0005,word_0006,word_0007,word_0008,...,word_1424,word_1425,word_1426,word_1427,word_1428,word_1429,word_1430,word_1431,word_1432,label
0,31336,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Neural_Networks
1,1061127,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Rule_Learning
2,1106406,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
3,13195,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
4,37879,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Probabilistic_Methods


In [6]:
cites_df.head()

,target,source
0,35,1033
1,35,103482
2,35,103515
3,35,1050679
4,35,1103960


# Unpacking the data

Now we unpack the data into more efficient data structures.

## Nodes and edges
The next code cell builds lists for the nodes and edges.

In [7]:
# list of nodes
nodes = content_df['paper_id'].to_list()

# list of edges
edges = cites_df.values.tolist()

print(f'There are {len(nodes)} nodes.')
print('  ' + ', '.join([str(v) for v in nodes[:5]]) + ', ...')
print()
print(f'There are {len(edges)} edges.')
print('  ' + ', '.join([str(e) for e in edges[:5]]) + ', ...')

There are 2485 nodes.
  31336, 1061127, 1106406, 13195, 37879, ...

There are 5209 edges.
  [35, 1033], [35, 103482], [35, 103515], [35, 1050679], [35, 1103960], ...


## Labels

Each node $u\in V$ has an associated label $\ell_u\in L$, where $L$ is the label set. (I've updated to code to use Numpy, as pointed out in class.)

In [8]:
# find the unique labels
label_set = np.unique(content_df['label'])

print(f'There are {len(label_set)} unique labels.')
for i,l in enumerate(label_set):
    print(f'  {i} -> {l}')

There are 7 unique labels.
  0 -> Case_Based
  1 -> Genetic_Algorithms
  2 -> Neural_Networks
  3 -> Probabilistic_Methods
  4 -> Reinforcement_Learning
  5 -> Rule_Learning
  6 -> Theory


Instead of storing the text label for each node, we decided to store the corresponding index. We can do that by creating a dictionary that maps from each label to its corresponding index. (Essentially inverting the index-to-label map implicit in the label_set list.)

In [9]:
# dictionary : label -> index
label_map = { l : i for i,l in enumerate(label_set)}

print('Label map:')
for l in label_map:
    print(f'  {l:22s} -> {label_map[l]}')

Label map:
  Case_Based             -> 0
  Genetic_Algorithms     -> 1
  Neural_Networks        -> 2
  Probabilistic_Methods  -> 3
  Reinforcement_Learning -> 4
  Rule_Learning          -> 5
  Theory                 -> 6


Now, we could make a dictionary mapping nodes to label (index), as follows.

In [10]:
# dictionary : node -> label index
labels = {u : x for u,x in zip(nodes, content_df['label'].map(lambda l : label_map[l]).values)}

for u in nodes[:5]:
    print(f'Node {u} has label index {labels[u]}.')

Node 31336 has label index 2.
Node 1061127 has label index 5.
Node 1106406 has label index 4.
Node 13195 has label index 4.
Node 37879 has label index 3.


However, for our purposes (vector operations), it is more efficient to store the labels as a vector $y\in\mathbb{R}^n$ and use a dictionary to match the nodes to the corresponding index.

In [11]:
# dictionary : node -> row index in feature matrix
node_map = {u : i for i,u in enumerate(nodes)}

# label vector
y = content_df['label'].map(lambda l : label_map[l]).values

for u in nodes[:5]:
    print(f'Node {u} has label index {y[node_map[u]]}.')

Node 31336 has label index 2.
Node 1061127 has label index 5.
Node 1106406 has label index 4.
Node 13195 has label index 4.
Node 37879 has label index 3.


## Features

Next, we need to keep track of the feature vector $x_u$ for each node $u\in V$. Our first iteration was to use a dictionary mapping $V$ to $\mathbb{R}^d$.

In [12]:
# dictionary : node -> feature vector
features = {u : x for u,x in zip(nodes, content_df.iloc[:,1:-1].values)}

for u in nodes[:5]:
    print(f'Node {u} contains {sum(features[u])} of the key words.')

Node 31336 contains 20 of the key words.
Node 1061127 contains 17 of the key words.
Node 1106406 contains 22 of the key words.
Node 13195 contains 21 of the key words.
Node 37879 contains 23 of the key words.


Again, this is not particularly efficient for some of the things we will need to do (e.g., matrix multiplication). So, instead, we'll store the feature vectors as rows in a $n\times d$ matrix and use the node map to index into it.

In [13]:
# feature matrix
X = content_df.iloc[:,1:-1].values.astype(float)     # converted to float in anticpation of using pytorch

for u in nodes[:5]:
    print(f'Node {u} maps to index {node_map[u]} and contains {sum(X[node_map[u],:])} of the key words.')

Node 31336 maps to index 0 and contains 20.0 of the key words.
Node 1061127 maps to index 1 and contains 17.0 of the key words.
Node 1106406 maps to index 2 and contains 22.0 of the key words.
Node 13195 maps to index 3 and contains 21.0 of the key words.
Node 37879 maps to index 4 and contains 23.0 of the key words.


# GCN by hand

In this section, we'll derive the graph convolutional network (GCN) from the following paper.

> Kipf, T.N., Welling, M., 2017. Semi-Supervised Classification with Graph Convolutional Networks. https://doi.org/10.48550/arXiv.1609.02907

The basic idea is to perform linear transformations of the form

$$
f(X)_u = W^T\left(\sum_{v\in\mathcal{N}_u}\frac{1}{\sqrt{d_ud_v}}x_v\right)+b.
$$

A few important points:
- The graph is modified to include self-loops. (Essentially, this replaces the adjacency matrix $A$ by $A+I$.)
- $d_u$ and $d_v$ are the degrees of the vertices $u$ and $v$ respectively (including counting the new self-loop).

## Storing the data in pytorch tensors

For this example, I subtracted the average from each feature vector so that each feature vector has average 0.

In [14]:
# pytorch feature tensor
X_pt = torch.from_numpy(X - np.average(X,axis=1)[:,None])

# pytorch label tensor
y_pt = torch.from_numpy(y)

## First (slow) attempt

Here's a fairly straightforward implementation of the formula. The weigths, $W$, and biases, $b$, are included by attaching a PyTorch <tt>Linear</tt> layer.

In [15]:
class SlowGCNLayer(nn.Module):
    
    def __init__(self, nodes, edges, in_channels, out_channels):
        
        # call the nn.Module initialization routine
        super().__init__()
        
        # attach basic data
        self.nodes = nodes
        self.edges = edges
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        # dictionary : node_id -> index
        self.node_map = {u : i for i,u in enumerate(nodes)}
        
        # linear operator holding the weights and biases
        self.L = nn.Linear(in_channels, out_channels)
        
        # initialize a neighbor dictionary with self-loops
        self.neighbors = { u : set([u]) for u in nodes}
        
        # add other neighbors from the edges list
        for u,v in edges:
            self.neighbors[u].add(v)
            self.neighbors[v].add(u)

    def degree(self, u):
        return len(self.neighbors[u])
    
    def forward(self, X):
        
        # tensor to accumulate the sum over neighbors
        fX = torch.zeros_like(X)
        
        # loop over all nodes
        for u in self.nodes:
            
            # gather node index and degree
            u_ind = self.node_map[u]
            u_deg = self.degree(u)
            
            # loop over all neighbors
            for v in self.neighbors[u]:
                
                # gather node index and degree
                v_ind = self.node_map[v]
                v_deg = self.degree(v)
                
                # update u's row in the summation
                fX[u_ind,:] += X[v_ind,:]/np.sqrt(u_deg*v_deg)

        # apply the linear operator to the sum and return
        return self.L(fX)

Here's an example that creates a GCN layer with 10 output channels.

In [16]:
in_channels = X.shape[1]
out_channels = 10
L1 = SlowGCNLayer(nodes, edges, in_channels, out_channels)

L1(X_pt).shape

torch.Size([2485, 10])

<tt>for</tt> loops in Python tend to be slow. Nested <tt>for</tt> loops are even slower. Knowing that, we shouldn't expect our current GCN implementation to be particuarly efficient.

In [17]:
%timeit L1(X_pt)

858 ms ± 72.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Second (faster) attempt

As discussed in class, the operation inside the summation is equivalent to left-multiplication of the data by the matrix $T=D^{-\frac{1}{2}}AD^{-\frac{1}{2}}$. Linear algebra operations tend to be faster in Python libraries like Numpy and PyTorch because these libraries include compiled implementations that are much more efficient than Python's for loops. The code cell below sets up the $T$ matrix.

In [18]:
# dimension of the adjacency matrix
num_nodes = len(nodes)

# initialize with self-loops
A = np.eye(num_nodes)

# add entries based on edge list
for u,v in edges:
    u_ind = node_map[u]
    v_ind = node_map[v]
    A[u_ind,v_ind] = 1
    A[v_ind,u_ind] = 1

# create the diagonal matrix D^{-1/2} 
DD = np.diag(1/np.sqrt(A.sum(axis=1)))

# create the T matrix
T = DD@A@DD

# convert to a sparse PyTorch tensor
T_pt = torch.from_numpy(T).to_sparse_csr()

/tmp/ipykernel_236/1417422144.py:21: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  T_pt = torch.from_numpy(T).to_sparse_csr()


Here's an updated implementation of the GCN layer using the $T$ matrix.

In [19]:
class GCNLayer(nn.Module):
    
    def __init__(self, T, in_channels, out_channels):
        
        # call the nn.Module initialization routine
        super().__init__()
        
        # attach basic data
        self.T = T
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        # linear operator holding the weights and biases
        self.L = nn.Linear(in_channels, out_channels)
        
    def forward(self, X):
        
        # apply T and the linear operator
        return self.L(self.T@X)

To compare the two implementations, we can create a member of the new class and copy the weights and biases from the original.

In [20]:
L2 = GCNLayer(T_pt, in_channels, out_channels)

L2.load_state_dict(L1.state_dict())

<All keys matched successfully>

As a simple test of our code, we can check if the two layers give the same output.

In [21]:
with torch.no_grad():
    print(torch.norm(L1(X_pt) - L2(X_pt)).item())

4.270169654765694e-15


We can also compare the evaluation time required to the previous implementation. The implementation using matrix multiplication should be 10-20 times faster.

In [22]:
%timeit L2(X_pt)

26.2 ms ± 2.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Putting the 'N' in GCN

Now it's time to build a network out of these layers.

In [23]:
hidden_channels = 256
hidden_layers = 4
out_channels = len(label_set)

class GCN(nn.Module):
    
    def __init__(self, T, in_channels, out_channels, hidden_channels, hidden_layers):
        
        # super-class initializer
        super().__init__()
        
        # create a module list for the layers
        self.layers = nn.ModuleList()
        
        # first hidden layer : in_channels -> hidden_channels
        self.layers.append(GCNLayer(T, in_channels, hidden_channels))
        
        # remaining hidden layers : hidden_channels -> hidden_channels
        for i in range(hidden_layers-1):
            self.layers.append(GCNLayer(T, hidden_channels, hidden_channels))
            
        # output layer : hidden_channels -> out_channels
        self.layers.append(GCNLayer(T, hidden_channels, out_channels))
        

    def forward(self, X):
        
        dropout = nn.functional.dropout
        
        l = self.layers[0]
        X = l(X).relu()
        X = dropout(X, 0.75, training=self.training)
        
        for l in self.layers[1:-1]:
            X = X + l(X).relu()

        X = dropout(X, 0.75, training=self.training)
        l = self.layers[-1]
        return l(X)
    
model = GCN(T_pt, in_channels, out_channels, hidden_channels, hidden_layers)

## Training and testing data

Here we split the data into training and testing sets.

In [24]:
num_nodes = len(nodes)
train_ind, test_ind = train_test_split(range(num_nodes), test_size=1000, stratify=y)

print(f'train: {len(train_ind)}')
print(f'test:  {len(test_ind)}')

train: 1485
test:  1000


## Training the model

In [25]:
lr = 3e-4
weight_decay = 4.5e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
loss_fn = nn.CrossEntropyLoss()

epochs = 101
output_every = 10

num_train = len(train_ind)
num_test = len(test_ind)

for t in range(epochs):
    
    model.train()
    y_hat = model(X_pt)
    loss = loss_fn(y_hat[train_ind], y_pt[train_ind])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if t % output_every == 0:
        
        model.eval()
        with torch.no_grad():
        
            y_hat = model(X_pt)
            train_acc = torch.sum(y_hat[train_ind].max(1).indices == y_pt[train_ind])/num_train
            test_acc = torch.sum(y_hat[test_ind].max(1).indices == y_pt[test_ind])/num_test
            
            print(f'epoch {t:5d}  train loss {loss.item()/num_train: 0.5e}  train acc {100*train_acc:6.2f}%  test acc {100*test_acc:6.2f}%')

epoch     0  train loss  1.29974e-03  train acc  17.98%  test acc  18.70%


epoch    10  train loss  1.22226e-03  train acc  29.23%  test acc  29.20%


epoch    20  train loss  1.12232e-03  train acc  29.23%  test acc  29.20%


epoch    30  train loss  9.50574e-04  train acc  65.25%  test acc  62.60%


epoch    40  train loss  6.82115e-04  train acc  76.50%  test acc  75.10%


...

epoch    60  train loss  3.34537e-04  train acc  87.61%  test acc  84.00%


epoch    70  train loss  2.57041e-04  train acc  90.44%  test acc  87.30%


epoch    80  train loss  2.18479e-04  train acc  91.25%  test acc  88.80%


epoch    90  train loss  1.98381e-04  train acc  91.85%  test acc  88.40%


epoch   100  train loss  1.77084e-04  train acc  92.32%  test acc  88.60%


# GCN using [PyG](https://www.pyg.org/)

In [26]:
edge_index = [ (node_map[u], node_map[v]) for u,v in edges]
edge_index += [ (node_map[u], node_map[v]) for v,u in edges]
edge_index = torch.tensor(edge_index).T

In [27]:
hidden_channels = 256
hidden_layers = 4
out_channels = len(label_set)

class GCN2(nn.Module):
    
    def __init__(self, edge_index, in_channels, out_channels, hidden_channels, hidden_layers):
        
        # super-class initializer
        super().__init__()
        
        self.edge_index = edge_index
        
        # create a module list for the layers
        self.layers = nn.ModuleList()
        
        # first hidden layer : in_channels -> hidden_channels
        self.layers.append(tgnn.GCNConv(in_channels, hidden_channels))
        
        # remaining hidden layers : hidden_channels -> hidden_channels
        for i in range(hidden_layers-1):
            self.layers.append(tgnn.GCNConv(hidden_channels, hidden_channels))
            
        # output layer : hidden_channels -> out_channels
        self.layers.append(tgnn.GCNConv(hidden_channels, out_channels))
        

    def forward(self, X):
        
        dropout = nn.functional.dropout
        
        l = self.layers[0]
        X = l(X, self.edge_index).relu()
        X = dropout(X, 0.75, training=self.training)
        
        for l in self.layers[1:-1]:
            X = X + l(X, self.edge_index).relu()

        X = dropout(X, 0.75, training=self.training)
        l = self.layers[-1]
        return l(X, self.edge_index)
    
model = GCN2(edge_index, in_channels, out_channels, hidden_channels, hidden_layers)

In [28]:
lr = 3e-4
weight_decay = 4.5e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
loss_fn = nn.CrossEntropyLoss()

epochs = 101
output_every = 10

num_train = len(train_ind)
num_test = len(test_ind)

for t in range(epochs):
    
    model.train()
    y_hat = model(X_pt)
    loss = loss_fn(y_hat[train_ind], y_pt[train_ind])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if t % output_every == 0:
        
        model.eval()
        with torch.no_grad():
        
            y_hat = model(X_pt)
            train_acc = torch.sum(y_hat[train_ind].max(1).indices == y_pt[train_ind])/num_train
            test_acc = torch.sum(y_hat[test_ind].max(1).indices == y_pt[test_ind])/num_test
            
            print(f'epoch {t:5d}  train loss {loss.item()/num_train: 0.5e}  train acc {100*train_acc:6.2f}%  test acc {100*test_acc:6.2f}%')

epoch     0  train loss  1.32430e-03  train acc  18.18%  test acc  17.50%


epoch    10  train loss  1.11990e-03  train acc  33.33%  test acc  32.40%


epoch    20  train loss  9.32468e-04  train acc  70.10%  test acc  67.80%


epoch    30  train loss  6.73592e-04  train acc  81.21%  test acc  79.80%


epoch    40  train loss  4.40061e-04  train acc  87.27%  test acc  85.30%


...

epoch    60  train loss  2.53249e-04  train acc  90.10%  test acc  88.80%


epoch    70  train loss  2.29370e-04  train acc  91.58%  test acc  89.00%


epoch    80  train loss  2.08358e-04  train acc  91.65%  test acc  88.80%


epoch    90  train loss  1.92950e-04  train acc  92.53%  test acc  88.90%


epoch   100  train loss  1.70246e-04  train acc  92.46%  test acc  88.90%
